In [1]:
import toml
from joint_calendar import generate_joint_ad_promo_schedule

config = toml.load("config.toml")

# Set up household parameters
household_sizes = config["household"]["household_sizes"]
household_size_distribution = config["household"]["household_size_distribution"]
base_consumption_rate = config["household"]["base_consumption_rate"]
pantry_min_percent = config["household"]["pantry_min_percent"]

# Set up retail environment
brand_list = list(config["brands"].keys())
brand_market_share = [
    config["brands"][brand]["current_market_share"] for brand in brand_list
]
try:
    assert round(sum(brand_market_share), 2) == 1.0
except AssertionError:
    print("Error: Brand market shares do not sum to 1.")


# Set up advertising and promotion
ad_channels = list(
    [config["brands"][brand]["advertising"]["channels"] for brand in brand_list]
)
joint_calendar = generate_joint_ad_promo_schedule(brand_list, config)

In [2]:
def retrieve_data_for_week(joint_calendar, week):
    return joint_calendar.loc[week, :]

In [3]:
retrieve_data_for_week(joint_calendar, 1)

A  price        2.5
   TV       1733.33
   Web            0
B  price        2.5
   TV          1200
   Web            0
Name: 1, dtype: object

In [4]:
def check_for_ads(brand, channel, week):
    return joint_calendar.loc[week, (brand, channel)]


In [5]:
check_for_ads("A", "TV", 1)

1733.33

In [6]:
def generate_brand_ad_channel_map(brand_list, config):
    brand_ad_channel_map = {}
    for brand in brand_list:
        brand_ad_channel_map[brand] = config["brands"][brand]["advertising"][
            "channels"
        ]
    return brand_ad_channel_map

In [7]:
brand_channel_map = generate_brand_ad_channel_map(brand_list, config)

In [8]:
brand_channel_map

{'A': ['TV', 'Web'], 'B': ['TV', 'Web']}

In [9]:
channel_set = list(set(channel for channels in brand_channel_map.values() for channel in channels))

In [10]:
for i in channel_set:
    print(i)

TV
Web


In [11]:
channel_priors = [0.6,0.4]

In [12]:
import random

def assign_weights(items, prior_weights):
    # Generate random fluctuations
    fluctuations = [random.random() for _ in items]
    
    # Apply fluctuations to prior weights
    weights = [w + f for w, f in zip(prior_weights, fluctuations)]
    
    # Normalize weights so they sum to 1
    weight_sum = sum(weights)
    weights = [w/weight_sum for w in weights]
    
    # Create a dictionary to map items to their weights
    weights_dict = dict(zip(items, weights))
    
    return weights_dict

In [13]:
channel_preferences = assign_weights(channel_set, channel_priors)

In [14]:
channel_preferences

{'TV': 0.5075334981214318, 'Web': 0.4924665018785682}

In [15]:
def calculate_adstock(week, joint_calendar, brand_channel_map, channel_preference):
    adstock = {}
    for brand, channels in brand_channel_map.items():
        for channel in channels:
            spend = joint_calendar.loc[week, (brand, channel)]
            weighted_spend = spend * channel_preference[channel]
            if brand in adstock:
                adstock[brand] += weighted_spend
            else:
                adstock[brand] = weighted_spend
    return adstock

In [16]:
calculate_adstock(1,joint_calendar, brand_channel_map, channel_preferences)

{'A': 879.7230382988213, 'B': 609.0401977457182}

In [17]:
def get_current_price(week, joint_calendar, brand):
    price = joint_calendar.loc[week, (brand, 'price')]
    return price


In [18]:
get_current_price(30, joint_calendar, 'A')

5.0

In [19]:
adstock = calculate_adstock(1,joint_calendar, brand_channel_map, channel_preferences)

In [20]:
def ad_decay(adstock, factor):
    return {brand: value / factor for brand, value in adstock.items()}


In [21]:
ad_decay(adstock, 2)

{'A': 439.86151914941064, 'B': 304.5200988728591}

In [22]:
def update_adstock(adstock1, adstock2):
    updated_adstock = adstock1.copy()
    for brand, value in adstock2.items():
        if brand in updated_adstock:
            updated_adstock[brand] += value
        else:
            updated_adstock[brand] = value
    return updated_adstock


In [23]:
adstock_2 = calculate_adstock(2,joint_calendar, brand_channel_map, channel_preferences)

In [24]:
adstock

{'A': 879.7230382988213, 'B': 609.0401977457182}

In [25]:
adstock_2


{'A': 879.7230382988213, 'B': 609.0401977457182}

In [26]:
update_adstock(adstock, adstock_2)

{'A': 1759.4460765976426, 'B': 1218.0803954914363}

In [27]:
def get_switch_probability(adstock, preferred_brand, default_loyalty_rate):
    brands = list(adstock.keys())
    adstock_values = list(adstock.values())
    
    if adstock[preferred_brand] > max(adstock_values):
        return {brand: 1 if brand == preferred_brand else 0 for brand in brands}
    
    elif sum(adstock_values) == 0:
        probabilities = {brand: default_loyalty_rate if brand == preferred_brand else (1-default_loyalty_rate)/(len(brands)-1) for brand in brands}
        return probabilities
    
    else:
        total_adstock = sum(adstock_values)
        probabilities = {brand: value/total_adstock for brand, value in adstock.items()}
        return probabilities



In [28]:
get_switch_probability(adstock, 'A', 0.1)

{'A': 0.5909086260325295, 'B': 0.4090913739674704}

In [33]:
adstock_test_hib = {'A': 0.0, 'B': 0.0}

In [34]:
get_switch_probability(adstock_test_hib, 'A', 0.1)

{'A': 0.1, 'B': 0.9}